In [22]:
import cv2 as cv 
import numpy as np
import mediapipe as mp 
import cvzone
from pynput.keyboard import Controller

In [9]:
# Mediapipe inits
mp_face_mesh = mp.solutions.face_mesh
LEFT_EYE =[ 362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385,384, 398 ]
RIGHT_EYE=[ 33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161 , 246 ] 
LEFT_IRIS = [474,475, 476, 477]
RIGHT_IRIS = [469, 470, 471, 472]

# Keyboard inits
keyboard_keys = [["Q", "W", "E", "R", "T", "Y", "U", "I", "O", "P"],
                  ["A", "S", "D", "F", "G", "H", "J", "K", "L", ";"],
                  ["Z", "X", "C", "V", "B", "N", "M", ",", ".", "/"]]

keyboard_keys = [['I', 'T'],
                 ['P', 'D', 'Y'],
                 ['R', 'BS']]

keyboard = Controller()
class Button():
    def __init__(self, pos, text, size=[85, 85]):
        self.pos = pos
        self.size = size
        self.text = text

buttons = []
for k in range(len(keyboard_keys)):
    for x, key in enumerate(keyboard_keys[k]):
        buttons.append(Button([100 * x + 25, 100 * k + 50], key))
        
def kb(img, buttons):
    for button in buttons:
        x, y = button.pos
        w, h = button.size
        cvzone.cornerRect(img, (button.pos[0], button.pos[1],
                                                   button.size[0],button.size[0]), 20 ,rt=0)
        cv.rectangle(img, button.pos, (int(x + w), int(y + h)), (255, 144, 30), cv.FILLED)
        cv.putText(img, button.text, (x + 20, y + 65),
                    cv.FONT_HERSHEY_PLAIN, 4, (0, 0, 0), 4)
    return img

In [23]:
# Attention
cap = cv.VideoCapture(0)
calibration = [False, False]
xydata = []

with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
) as face_mesh:
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv.flip(frame, 1)
        rgb_frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        h, w = frame.shape[:2]
        results = face_mesh.process(rgb_frame)
        # Draw irises
        if results.multi_face_landmarks:
            mesh_points=np.array([np.multiply([p.x, p.y], [w, h]).astype(float) for p in results.multi_face_landmarks[0].landmark])
            (l_cx, l_cy), l_radius = cv.minEnclosingCircle(mesh_points[LEFT_IRIS])
            (r_cx, r_cy), r_radius = cv.minEnclosingCircle(mesh_points[RIGHT_IRIS])
            print (l_cx)
            break
            center_left = np.array([l_cx, l_cy], dtype=np.int32)
            center_right = np.array([r_cx, r_cy], dtype=np.int32)
            cv.circle(frame, center_left, int(l_radius), (255,0,255), 1, cv.LINE_AA)
            cv.circle(frame, center_right, int(r_radius), (255,0,255), 1, cv.LINE_AA)
        
        key = cv.waitKey(1)
        # Calibrate top-left 
        if key==ord('e'):
            cv.rectangle(frame, (0,0), (50,50), (0,0,255), -1)
            #calibration_top_left = center_left
            calibration_top_left = [(l_cx+r_cx)/2, (l_cy+r_cy)/2]
            calibration[0] = True
        # Calibrate bottom-right
        if key==ord('r'):
            cv.rectangle(frame, (w-50, h-50), (w,h), (255,0,0), -1)
            #calibration_bottom_right = center_left
            calibration_bottom_right = [(l_cx+r_cx)/2, (l_cy+r_cy)/2]
            calibration[1] = True
        # Relying on an average coordinates of eyes (draw circle manually)
        # Is it really linear? range is ~10pixels
        if key==ord('d'):
            pos = [(l_cx+r_cx)/2, (l_cy+ r_cy)/2]
            if all(calibration):
                t_l = calibration_top_left
                b_r = calibration_bottom_right
                x = (pos[0]-t_l[0])/(b_r[0]-t_l[0])*w
                y = (pos[1]-t_l[1])/(b_r[1]-t_l[1])*h
                cv.circle(frame, (int(x),int(y)), 40, (255,0,0), -1)
        # Dont think it's good
        pos = [(l_cx+r_cx)/2, (l_cy+r_cy)/2]
        if all(calibration):
            t_l = calibration_top_left
            b_r = calibration_bottom_right
            x = (pos[0]-t_l[0])/(b_r[0]-t_l[0])*w
            y = (pos[1]-t_l[1])/(b_r[1]-t_l[1])*h
            xydata.append([x,y])
        # Smooth a bit and draw
        if len(xydata)>6:
            x = sum([t[0] for t in xydata[-6:]])/6
            y = sum([t[1] for t in xydata[-6:]])/6
            cv.circle(frame, (int(x),int(y)), 
                    int(max(
                        np.sqrt(np.var([t[0] for t in xydata[-6:]])),
                        np.sqrt(np.var([t[1] for t in xydata[-6:]]))
                        )), 
                    (255,0,0), -1)
        # Rescale
        aspect_ratio = h/w
        resized_h = 950
        resized_w = int(resized_h/aspect_ratio)
        frame = cv.resize(frame, (resized_w, resized_h))
        cv.imshow('img', frame)
        
        if key==ord('q'):
            break
        
cap.release()
cv.destroyAllWindows()

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\shapedescr.cpp:201: error: (-215:Assertion failed) count >= 0 && (depth == CV_32F || depth == CV_32S) in function 'cv::minEnclosingCircle'


In [6]:
# Keyboard stuff
cap = cv.VideoCapture(0)
calibration = [False, False]
xydata = []
text = ''

with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
) as face_mesh:
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv.flip(frame, 1)
        rgb_frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        h, w = frame.shape[:2]
        results = face_mesh.process(rgb_frame)
        if results.multi_face_landmarks:
            mesh_points=np.array([np.multiply([p.x, p.y], [w, h]).astype(int) for p in results.multi_face_landmarks[0].landmark])
            (l_cx, l_cy), l_radius = cv.minEnclosingCircle(mesh_points[LEFT_IRIS])
            (r_cx, r_cy), r_radius = cv.minEnclosingCircle(mesh_points[RIGHT_IRIS])
            center_left = np.array([l_cx, l_cy], dtype=np.int32)
            center_right = np.array([r_cx, r_cy], dtype=np.int32)
            cv.circle(frame, center_left, int(l_radius), (255,0,255), 1, cv.LINE_AA)
            cv.circle(frame, center_right, int(r_radius), (255,0,255), 1, cv.LINE_AA)
        
        key = cv.waitKey(1)
        if key==ord('e'):
            cv.rectangle(frame, (0,0), (50,50), (0,0,255), -1)
            #calibration_top_left = center_left
            calibration_top_left = [(l_cx+r_cx)/2, (l_cy+r_cy)/2]
            calibration[0] = True

        if key==ord('r'):
            cv.rectangle(frame, (w-50, h-50), (w,h), (255,0,0), -1)
            #calibration_bottom_right = center_left
            calibration_bottom_right = [(l_cx+r_cx)/2, (l_cy+r_cy)/2]
            calibration[1] = True

        if key==ord('d'):
            pos = [(l_cx+r_cx)/2, (l_cy+ r_cy)/2]
            if all(calibration):
                t_l = calibration_top_left
                b_r = calibration_bottom_right
                x = (pos[0]-t_l[0])/(b_r[0]-t_l[0])*w
                y = (pos[1]-t_l[1])/(b_r[1]-t_l[1])*h
                cv.circle(frame, (int(x),int(y)), 40, (255,0,0), -1)
        
        pos = [(l_cx+r_cx)/2, (l_cy+r_cy)/2]
        if all(calibration):
            t_l = calibration_top_left
            b_r = calibration_bottom_right
            x = (pos[0]-t_l[0])/(b_r[0]-t_l[0])*w
            y = (pos[1]-t_l[1])/(b_r[1]-t_l[1])*h
            xydata.append([x,y])
            
        font = cv.FONT_HERSHEY_SIMPLEX
        frame = kb(frame, buttons)
        # Press a button to kick the letter in
        if key == ord('s'):
            if len(xydata)>25:
                x = sum([t[0] for t in xydata[-10:]])/10
                y = sum([t[1] for t in xydata[-10:]])/10
                for button in buttons:
                    bx, by = button.pos
                    bw, bh = button.size
                    if bx< x < bx+bw and by < y < by+bh:
                        if button.text=='BS':
                            text=text[:-1]
                        else:
                            text = text + button.text
        frame = cv.putText(frame, text, (300,400), font, 2, (255,255,255))
        aspect_ratio = h/w
        resized_h = 950
        resized_w = int(resized_h/aspect_ratio)
        frame = cv.resize(frame, (resized_w, resized_h))
        cv.imshow('img', frame)
        
        if key==ord('q'):
            break
        
cap.release()
cv.destroyAllWindows()